In [1]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import dask.multiprocessing

### Cliente DASK

In [2]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57602,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:57639,Total threads: 2
Dashboard: http://127.0.0.1:57640/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:57606,


In [23]:
# Foi necessário colocar dtype em todas as colunas pois o dask estava inferindo muito mal o tipo de cada coluna,
# dificultando muito a análise. Também foi necessário inferir todos os inteiros e doubles como strings.

reviews = dd.read_csv('steam_reviews.csv',
                       sep=',',
                       encoding='UTF-8',
                      engine='python',
                      on_bad_lines='skip',
                       dtype={'Unnamed: 0': 'str',
                              'app_id': 'str',
                              'app_name': 'str',
                              'review_id': 'str',
                              'language': 'str',
                              'review': 'str',
                              'timestamp_created': 'str',
                              'timestamp_updated': 'str',
                              'recommended': 'str',
                              'votes_helpful': 'str',
                              'votes_funny': 'str',
                              'weighted_vote_score': 'str',
                              'comment_count': 'str',
                              'steam_purchase': 'str',
                              'received_for_free': 'str',
                              'written_during_early_access': 'str',
                              'author.steamid': 'str',
                              'author.num_games_owned': 'str',
                              'author.num_reviews': 'str',
                              'author.playtime_forever': 'str',
                              'author.playtime_last_two_weeks': 'str',
                              'author.playtime_at_review': 'str',
                              'author.last_played': 'str'})

In [24]:
reviews.head()

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1611383744.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0


In [25]:
linhas = len(reviews)

print(f'O dataset possui {linhas} de linhas!')

O dataset possui 21756295 de linhas!


In [26]:
# Renomeando a coluna de índice

columns = reviews.columns.to_list()
columns[0] = "index"
cols_dict = {}

for idx, column in enumerate(reviews.columns.to_list()):
    cols_dict[column] = columns[idx]

reviews = reviews.rename(columns=cols_dict)

In [27]:
reviews.head()

,index,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1611383744.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0


Estamos tendo muita dificuldade para limpar os dados, para essa entrega não foi possível limpar 100% do jeito que gostaríamos e por isso deixamos em branco, mas a limpeza acontecerá! :)